# Generate webpage content for calculation_system_relax records

This Notebook is designed for reading finished calculation_system_relax records and generating the associated webpage content.

#### Library imports

In [1]:
# Standard Python libraries
from __future__ import print_function
import glob
import os
from collections import OrderedDict
from copy import deepcopy

from IPython.core.display import display, HTML

# pandas.pydata.org
import pandas as pd

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/DataModelDict
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

import analysis

#### Plotting library imports

In [2]:
# https://bokeh.pydata.org/
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.resources import Resources, CDN
from bokeh.io import output_notebook
from bokeh.models import Range1d
print('bokeh version =', bokeh.__version__)
output_notebook()

bokeh version = 0.12.7


Loading BokehJS ...

## 1. Read Calculation Data

This section reads in raw data from a database. 

## 1. Raw Data

This section reads in or generates the raw_data associated with the calculation. 

### 1.1 Initialize database

- __dbasename__ is used here to predefine different dbase settings
- __dbase__ is the iprPy.Database object to use for accessing a database

In [3]:
dbasename = 'iprhub'

# 'local' is a local directory
if   dbasename == 'local':
    dbase = iprPy.Database('local',   host='C:\Users\lmh1\Documents\calculations\ipr\library')

# 'test' is a local directory for testing 
if   dbasename == 'test':
    dbase = iprPy.Database('local',   host='C:\Users\lmh1\Documents\calculations\ipr\library_test')
    
# 'curator' is a local MDCS curator
elif dbasename == 'curator':
    dbase = iprPy.Database('curator', host='http://127.0.0.1:8000/', 
                                      user='admin', 
                                      pswd='admin')

# 'iprhub' is the remote MDCS curator at iprhub
elif dbasename == 'iprhub':
    dbase = iprPy.Database('curator', host='https://iprhub.nist.gov/', 
                                      user='lmh1',
                                      pswd='C:/users/lmh1/documents/iprhub/iprhub_password.txt',
                                      cert='C:/users/lmh1/documents/iprhub/iprhub-ca.pem')
else:
    raise ValueError('unknown dbasename ' + dbasename)

### 1.2 Access records

In [4]:
proto_df = dbase.get_records_df(style='crystal_prototype')
print(str(len(proto_df)) + ' prototype records loaded')

19 prototype records loaded


In [5]:
pot_df = dbase.get_records_df(style='potential_LAMMPS')
print(str(len(pot_df)) + ' potential records loaded')

156 potential records loaded


In [6]:
raw_df = dbase.get_records_df(style='calculation_system_relax')
print(str(len(raw_df)) + ' calculation records loaded')

c:\users\lmh1\documents\python-packages\atomman\atomman\core\ElasticConstants.py:93: RuntimeWarning: divide by zero encountered in true_divide
  value[np.isclose(value/value.max(), 0.0, atol=1e-9)] = 0.0
c:\users\lmh1\documents\python-packages\atomman\atomman\core\ElasticConstants.py:93: RuntimeWarning: invalid value encountered in true_divide
  value[np.isclose(value/value.max(), 0.0, atol=1e-9)] = 0.0


19546 calculation records loaded


### 1.3 Check errors

In [7]:
if 'error' in raw_df:
    for error in np.unique(raw_df[pd.notnull(raw_df.error)].error):
        print(error)
        print()

Traceback (most recent call last):
  File "calc_LAMMPS_ELASTIC.py", line 284, in <module>
    main(*sys.argv[1:]) 
  File "calc_LAMMPS_ELASTIC.py", line 58, in main
    pressure_unit = input_dict['pressure_unit'])
  File "calc_LAMMPS_ELASTIC.py", line 103, in lammps_ELASTIC_refine
    dmax=dmax, pressure_unit=pressure_unit)
  File "calc_LAMMPS_ELASTIC.py", line 186, in lammps_ELASTIC
    output = lmp.run(lammps_command, 'in.elastic', mpi_command)
un.py", line 98, in run
    raise ValueError('Invalid LAMMPS input: 
%s' % lines[-2])
ValueError: Invalid LAMMPS input: 
ERROR: Did not assign all restart atoms correctly (../read_restart.cpp:513)

Traceback (most recent call last):
  File "calc_LAMMPS_ELASTIC.py", line 284, in <module>
    main(*sys.argv[1:]) 
  File "calc_LAMMPS_ELASTIC.py", line 58, in main
    pressure_unit = input_dict['pressure_unit'])
  File "calc_LAMMPS_ELASTIC.py", line 115, in lammps_ELASTIC_refine
    dmax=dmax, pressure_unit=pressure_unit)
  File "calc_LAMMPS_ELAST

## 2. Process Data

This section processes and refines the data.

### 2.1 Identify composition

We need to identify the composition of each calculation so that we can collect duplicates and filter out artificial compounds.

- __counts__ is a dictionary counting the number of times each atype appears in a crystal prototype's unit cell (i.e. the number of symmetry equivalent sites)

In [9]:
counts = {}
for i, prototype in proto_df.iterrows():
    model = DM(dbase.get_record(name=prototype.id, style='crystal_prototype').content)
    counts[prototype.id] = np.unique(model.finds('component'), return_counts=True)[1]

- __comp_refine()__ takes a list of symbols and count of how many times each symbol appears in a structure and generates a composition string.__comp_refine__ takes a list of symbols and count of how many times each symbol appears in a structure and generates a composition string.

In [10]:
def comp_refine(symbols, counts):
    """Takes a list of symbols and count of how many times each symbol appears and generates a composition string."""
    primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47]
    
    sym_dict = {}
    for i in xrange(len(symbols)):
        sym_dict[symbols[i]] = counts[i]
    
    for prime in primes:
        if max(sym_dict.values()) < prime:
            break
        
        while True:
            breaktime = False
            for value in sym_dict.values():
                if value % prime != 0:
                    breaktime = True
                    break
            if breaktime:
                break
            for key in sym_dict:
                sym_dict[key] /= prime
    
    composition=''
    for key in sorted(sym_dict):
        if sym_dict[key] > 0:
            composition += key
            if sym_dict[key] != 1:
                composition += str(sym_dict[key])
            
    return composition       

In [11]:
compositions = []
for i, calc in raw_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.family]))
raw_df = raw_df.assign(composition=compositions)

### 2.2 Identify current ipr potentials 

In [12]:
# Extract versionstyle and versionnumber from potential implementation ids
versionstyle = []
versionnumber = []
for name in pot_df['id'].values:
    version = name.split('--')[-1]
    try:
        versionnumber.append(int(version[-1]))
    except:
        versionnumber.append(np.nan)
        versionstyle.append(version)
    else:
        versionstyle.append(version[:-1])

pot_df['versionstyle'] = versionstyle
pot_df['versionnumber'] = versionnumber

# Loop through unique potential id's
includeid = []
for pot_id in np.unique(pot_df.pot_id.values):
    check_df = pot_df[pot_df.pot_id == pot_id]
    check_df = check_df[check_df.versionstyle == 'ipr']
    check_df = check_df[check_df.versionnumber == check_df.versionnumber.max()]
    if len(check_df) == 1:
        includeid.append(check_df['id'].values[0])
    elif len(check_df) > 1:
        raise ValueError('Bad currentIPR check for '+pot_id)

# Identify current IPR potentials
raw_df['currentIPR'] = raw_df.potential_LAMMPS_id.isin(includeid)

### 2.3 Identify crystals that have relaxed to a different crystal family

In [13]:
tol = 1e-5

In [14]:
crystal_families = {}
for proto_id, crystal_family in zip(proto_df['id'], proto_df.crystal_family):
    crystal_families[proto_id] = crystal_family
    
crystal_families = {
    #elemental
    'A1--Cu--fcc':                'cubic',
    'A2--W--bcc':                 'cubic',
    'A3--Mg--hcp':                'hexagonal',
    'A3\'--alpha-La--double-hcp': 'hexagonal',
    'A4--C--dc':                  'cubic',
    'A5--beta-Sn':                'tetragonal',
    'A6--In--bct':                'tetragonal',
    'A7--alpha-As':               'hexagonal',
    'A15--beta-W':                'cubic',
    'Ah--alpha-Po--sc':           'cubic',
   #1:1
    'B1--NaCl--rock-salt':        'cubic',
    'B2--CsCl':                   'cubic',
    'B3--ZnS--cubic-zinc-blende': 'cubic',
    'L1_0--AuCu':                 'tetragonal',
   #1:2
    'C1--CaF2--fluorite':         'cubic',
   #1:3
    'A15--Cr3Si':                 'cubic',
    'D0_3--BiF3':                 'cubic',
    'L1_2--AuCu3':                'cubic',
   #1:1:2
    'L2_1--AlCu2Mn--heusler':     'cubic'
}
np.unique(crystal_families.values())

array(['cubic', 'hexagonal', 'tetragonal'], 
      dtype='|S10')

In [15]:
hasrelaxed = np.empty(len(raw_df), dtype=bool)
for i, row in enumerate(raw_df.itertuples()):
    
    # Extract data for comparison
    a = row.final_a
    b = row.final_b
    c = row.final_c
    family = row.family
    crystal_family = crystal_families[family]
    
    # Evaluate based on crystal_family
    if crystal_family == 'cubic':
        if np.isclose(b/a, 1.0, atol=tol, rtol=0.0) and np.isclose(c/a, 1.0, atol=tol, rtol=0.0):
            hasrelaxed[i] = False
        else:
            hasrelaxed[i] = True
 
    elif crystal_family == 'hexagonal':
        if np.isclose(b/a, 3.**0.5, atol=tol, rtol=0.0):
            hasrelaxed[i] = False
        else:
            hasrelaxed[i] = True
            
    elif crystal_family == 'tetragonal':
        if np.isclose(b/a, 1.0, atol=tol, rtol=0.0) and not np.isclose(c/a, 1.0, atol=tol, rtol=0.0):
            hasrelaxed[i] = False
        else:
            hasrelaxed[i] = True

    # Evaluate based on family
    if family == 'A6--In--bct':
        if np.isclose(c/a, 2**(0.5), atol=tol, rtol=0.0):
            hasrelaxed[i] = True
    elif family == 'L1_0--AuCu':
        if np.isclose(c/a, 2**(0.5)/2, atol=tol, rtol=0.0):
            hasrelaxed[i] = True
            
raw_df['hasrelaxed'] = hasrelaxed

### 2.4 Remove unwanted calculations

Here is where we filter out unwanted entries (i.e. rows).

- __df__ is the dataframe during/after processing and refining

In [16]:
raw_df.keys()

Index([u'C', u'E_cohesive', u'LAMMPS_version', u'calc_key', u'calc_script',
       u'error', u'family', u'final_a', u'final_b', u'final_c', u'initial_a',
       u'initial_b', u'initial_c', u'iprPy_version', u'load_file',
       u'load_options', u'load_style', u'potential_LAMMPS_id',
       u'potential_LAMMPS_key', u'potential_id', u'potential_key',
       u'pressure_xx', u'pressure_yy', u'pressure_zz', u'sizemults', u'status',
       u'strainrange', u'symbols', u'temperature', u'composition',
       u'currentIPR', u'hasrelaxed'],
      dtype='object')

In [17]:
df = deepcopy(raw_df)

# Ignore unfinished or error calculations
df = df[df.status == 'finished']

# Ignore any implementations that are not current IPR implementations
df = df[df.currentIPR == True]

# Ignore any calculations that relaxed to a different crystal family
df = df[df.hasrelaxed == False]

# Ignore any that don't use the standard run parameters
df = df[np.isclose(df.strainrange, 1e-8, atol=1e-10, rtol=0.0) 
      & np.isclose(df.pressure_xx, 0.0, atol=1e-10, rtol=0.0)
      & np.isclose(df.pressure_yy, 0.0, atol=1e-10, rtol=0.0)
      & np.isclose(df.pressure_zz, 0.0, atol=1e-10, rtol=0.0)]

# Ignore false compounds (where # of unique symbols != # of symbols)
df = df[df.symbols.apply(lambda x: len(np.unique(x))) == df.symbols.apply(lambda x: len(x))] 

# Ignore duplicate compounds
ignore = set()
for i in xrange(len(df)):
    trunc = df.iloc[i+1:]
    matches = trunc.calc_key[  (trunc.potential_id == df.iloc[i].potential_id) 
                             & (trunc.family == df.iloc[i].family) 
                             & (trunc.composition == df.iloc[i].composition)
                             & (trunc.calc_script == df.iloc[i].calc_script)
                             & np.isclose(trunc.E_cohesive, df.iloc[i].E_cohesive, atol=1e-6, rtol=0.0)
                             & np.isclose(trunc.final_a, df.iloc[i].final_a, atol=1e-6, rtol=0.0)
                             & np.isclose(trunc.final_b, df.iloc[i].final_b, atol=1e-6, rtol=0.0)
                             & np.isclose(trunc.final_c, df.iloc[i].final_c, atol=1e-6, rtol=0.0)
                            ].tolist()
    ignore = ignore.union(matches)
df = df[~df.calc_key.isin(ignore)]

df.reset_index(drop=True, inplace=True)
print(str(len(df)) + ' records after filtering')

6389 records after filtering


### 2.5 Filter out extra data

Here, we limit the DataFrame to only the data that we care about (i.e. columns).

- __headers__ gives the list of data columns from raw_data to include in and how they should be renamed in data.

In [18]:
#                        raw names       new names
headers = OrderedDict([ ('calc_script',  'calculation'),
                        ('potential_id', 'potential'  ),
                        ('family',       'family'     ),
                        ('composition',  'composition'),
                        ('E_cohesive',   'Ecoh'),
                        ('final_a',      'alat'),
                        ('final_b',      'blat'),
                        ('final_c',      'clat'),
                        ('C',            'C'),
                      ])

df = pd.DataFrame(df, columns=headers.keys())
df.rename(columns=headers, inplace=True)
df

,calculation,potential,family,composition,Ecoh,alat,blat,clat,C
0,calc_refine_structure,2013--Smirnova-D-E--U-Mo-Xe,A6--In--bct,Xe,0.001966,6.031811,6.031811,8.954321,[[ -4.63377932e-02 2.31778108e-02 3.787362...
1,calc_refine_structure,2014--Bonny-G--W-H-He-2,A1--Cu--fcc,W,-8.527178,3.792585,3.792585,3.792585,[[ 3.90949397 3.08798085 3.08798085 0. ...
2,calc_refine_structure,2013--Zhou-X-W--Zn-Cd-Hg-S-Se-Te,C1--CaF2--fluorite,SeTe2,-1.720899,7.252711,7.252711,7.252711,[[ 1.65328025 1.07490581 1.07490581 0. ...
3,calc_LAMMPS_ELASTIC,2013--Bonny-G--Fe-Ni-Cr,A15--Cr3Si,FeNi3,-4.242726,4.521002,4.521002,4.521002,[[ -6.00876077e+05 -6.00876993e+05 -6.008769...
4,calc_refine_structure,2011--Bonny-G--Fe-Ni-Cr,A3'--alpha-La--double-hcp,Cr,-3.661201,2.526889,4.376700,8.522397,[[ 2.13171052 0.86460958 0.54638969 0. ...
5,calc_LAMMPS_ELASTIC,2012--Mendelev-M-I--Ni-Zr,A2--W--bcc,Ni,-4.294454,2.757031,2.757031,2.757031,[[ 0.62604479 0.56203089 0.56203089 0. ...
6,calc_refine_structure,2013--Smirnova-D-E--U-Mo-Xe,C1--CaF2--fluorite,Mo2U,-4.841783,6.259539,6.259539,6.259539,[[ 1.57229808 1.21066003 1.21066003 0. ...
7,calc_refine_structure,2013--Zhou-X-W--Zn-Cd-Hg-S-Se-Te,D0_3--BiF3,CdSe3,-2.025611,7.416059,7.416059,7.416059,[[ 1.99881816 2.03422934 2.03422934 0. ...
8,calc_refine_structure,2003--Hoyt-J-J--Cu-Pb,L1_2--AuCu3,Cu3Pb,-3.021148,4.004669,4.004669,4.004669,[[ 1.87169734 1.78179936 1.78179936 0. ...
9,calc_refine_structure,1987--Ackland-G-J--Cu,A4--C--dc,Cu,-2.250774,5.921937,5.921937,5.921937,[[ 0.65119877 0.38256871 0.38256871 0. ...


## 3. HTML Tables

This section takes the processed data and generates per_potential html tables.

In [19]:
html_info_file = 'html_info.html'
with open(html_info_file) as f:
    html_info = f.read()
display(HTML(html_info))

In [20]:
table_style_file = 'webtablestyle.html'

In [21]:
showSelectionScript = '<script type="text/javascript" src="analysis/showSelection.js"></script>\n'

In [22]:
with open(table_style_file) as f:
    table_style = f.read() 

In [23]:
def gen_struct_table(df, potential):
    
    headers = OrderedDict([ ('family', 'prototype'),
                            ('calculation', 'calculation'),
                            ('Ecoh', '<i>E</i><sub>coh</sub> (eV)'),
                            ('alat', '<i>a</i><sub>0</sub> (&Aring;)'),
                            ('blat', '<i>b</i><sub>0</sub> (&Aring;)'),
                            ('clat', '<i>c</i><sub>0</sub> (&Aring;)')])
    def float_fmt(value):
        return '%8.4f' % value
    
    pot_df = df[df.potential==potential]
    tables = OrderedDict()
    for composition in np.unique(pot_df.composition):
        table_df = pot_df[pot_df.composition==composition].sort_values('Ecoh')
        table_df = pd.DataFrame(table_df, columns=headers.keys())

        table_df.Ecoh = uc.get_in_units(table_df.Ecoh, 'eV')
        table_df.alat = uc.get_in_units(table_df.alat, 'Angstrom')
        table_df.blat = uc.get_in_units(table_df.blat, 'Angstrom')
        table_df.clat = uc.get_in_units(table_df.clat, 'Angstrom')
        table_df.calculation = [c[5:] for c in table_df.calculation]

        table_df.rename(columns=headers, inplace=True)
        table_df.reset_index(drop=True, inplace=True)
        tables[composition] = table_df.to_html(index=False, float_format=float_fmt, escape=False, classes='datatable')     
    
    return 'Select composition: ' + analysis.showSelection('Struct', tables)
    
    
def gen_Cij_table(df, potential):
    shift = {0:1, 1:2, 2:3, 3:4, 4:5, 5:6}
    def float_fmt(value):
        return '%.2f' % value
    
    pot_df = df[df.potential==potential]
    tables = OrderedDict()
    
    for composition in np.unique(pot_df.composition):
        comp_df = pot_df[pot_df.composition==composition]
        for family in np.unique(comp_df.family):
            fam_df = comp_df[comp_df.family==family]
            
            html = ''
            for i, row in fam_df.iterrows():
                Cij = uc.get_in_units(row.C.Cij, 'GPa')
                C_df = pd.DataFrame(Cij)
                C_df = C_df.rename(index=shift, columns=shift)

                html += '<i>C<sub>ij</sub></i> (GPa) [%s <i>a</i><sub>0</sub> = %.4f &Aring;] =' % (row.calculation[5:], row.alat)
                html += C_df.to_html(float_format=float_fmt, escape=False, classes='datatable')
            tables[composition + ' ' + family] = html + '<br>'
    
    return 'Select structure: ' + analysis.showSelection('Cij', tables)        

In [24]:
html = showSelectionScript+table_style
html += '<h3>Cohesive Energy and Lattice Constants</h3>\n'
html += gen_struct_table(df, '2009--Purja-Pun-G-P--Ni-Al')
html += '<h3> Elastic Constants</h3>\n'
html += gen_Cij_table(df, '2009--Purja-Pun-G-P--Ni-Al')

display(HTML(html))

prototype,calculation,Ecoh (eV),a0 (Å),b0 (Å),c0 (Å)
A1--Cu--fcc,LAMMPS_ELASTIC,-3.3600,4.0500,4.0500,4.0500
A1--Cu--fcc,refine_structure,-3.3600,4.0500,4.0500,4.0500
A15--beta-W,LAMMPS_ELASTIC,-3.3469,5.1467,5.1467,5.1467
A15--beta-W,refine_structure,-3.3469,5.1467,5.1467,5.1467
A3'--alpha-La--double-hcp,refine_structure,-3.3439,2.8458,4.9291,9.5344
A3'--alpha-La--double-hcp,LAMMPS_ELASTIC,-3.3439,2.8458,4.9291,9.5344
A3--Mg--hcp,refine_structure,-3.3322,2.8191,4.8828,4.9445
A3--Mg--hcp,LAMMPS_ELASTIC,-3.3322,2.8191,4.8828,4.9445
A2--W--bcc,refine_structure,-3.2545,3.2354,3.2354,3.2354
A5--beta-Sn,refine_structure,-3.0359,5.3506,5.3506,2.7773


## 4. Generate for all potentials

#### Generation parameters

- __savedir__ is the directory where the files will be saved.

In [25]:
savedir = '../webcontent/perpotential'

In [26]:
# Loop over all potentials
for potential in np.unique(df.potential):
    print(potential)

    # Generate html content
    html = html_info
    html += '<h3>Cohesive Energy and Lattice Constants</h3>\n'
    html += gen_struct_table(df, potential)
    html += '<h3> Elastic Constants</h3>\n'
    html += gen_Cij_table(df, potential)
    
    with open(os.path.join(savedir, potential, 'Structure.html'), 'w') as f:
        f.write(html)

1985--Foiles-S-M--Ni-Cu
1987--Ackland-G-J--Ag
1987--Ackland-G-J--Au
1987--Ackland-G-J--Cu
1987--Ackland-G-J--Mo
1987--Ackland-G-J--Ni
1989--Adams-J-B--Ag
1989--Adams-J-B--Au
1989--Adams-J-B--Cu
1989--Adams-J-B--Ni
1989--Adams-J-B--Pd
1989--Adams-J-B--Pt
1992--Ackland-G-J--Ti
1995--Angelo-J-E--Ni-Al-H
1996--Farkas-D--Nb-Ti-Al
1997--Ackland-G-J--Fe
1997--Liu-X-Y--Al-Mg
1998--Liu-X-Y--Al-Mg
1999--Liu-X-Y--Al-Cu
1999--Mishin-Y--Al
1999--Mishin-Y--Ni
2000--Landa-A--Al-Pb
2000--Sturgeon-J-B--Al
2001--Mishin-Y--Cu-1
2002--Mishin-Y--Ni-Al
2003--Han-S--Cs
2003--Han-S--K
2003--Han-S--Li
2003--Han-S--Na
2003--Han-S--Rb
2003--Han-S--V
2003--Han-S--W
2003--Hoyt-J-J--Cu-Pb
2003--Li-Y-H--Ta
2003--Mendelev-M-I--Fe-2
2003--Mendelev-M-I--Fe-5
2003--Zope-R-R--Al
2003--Zope-R-R--Ti-Al
2004--Ackland-G-J--Fe-P
2004--Liu-X-Y--Al
2004--Mishin-Y--Ni-Al
2004--Zhou-X-W--Ag
2004--Zhou-X-W--Al
2004--Zhou-X-W--Au
2004--Zhou-X-W--Co
2004--Zhou-X-W--Cu
2004--Zhou-X-W--Fe
2004--Zhou-X-W--Mg
2004--Zhou-X-W--Mo
2004--Zh